In [ ]:
import f1_2020_telemetry.packets

help(f1_2020_telemetry.packets)
import socket
import time
import json
import pickle
from f1_2020_telemetry.packets import unpack_udp_packet

udp_socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
udp_socket.bind(("", 20777))


L = []
js = []
start = time.time()
i= 0 
while True:
    udp_packet = udp_socket.recv(2048)
    packet = unpack_udp_packet(udp_packet)
    if isinstance(packet, f1_2020_telemetry.packets.PacketSessionData_V1):
        L.append((time.time(), packet))
        if i%10 ==0:
            with open("our_weather.pkl", 'wb') as f:
                pickle.dump(L, f)
    if time.time() - start > 1800:
        break

Help on module f1_2020_telemetry.packets in f1_2020_telemetry:

NAME
    f1_2020_telemetry.packets - F1 2020 UDP Telemetry support package

DESCRIPTION
    This package is based on the CodeMasters Forum post documenting the F1 2020 packet format:
    
        https://forums.codemasters.com/topic/54423-f1%C2%AE-2020-udp-specification/
    
    Compared to the definitions given there, the Python version has the following changes:
    
    (1) In the 'PacketMotionData' structure, the comments for the three m_angularAcceleration{X,Y,Z} fields erroneously
        refer to 'velocity' rather than 'acceleration'. This was corrected.
    (2) In the 'CarSetupData' structure, the comment of the m_rearAntiRollBar refer to rear instead of front. This was corrected.
    (3) In the Driver IDs table, driver 34 has name "Wilheim Kaufmann".
        This is a typo; whenever this driver is encountered in the game, his name is given as "Wilhelm Kaufmann".
    (4) In the 'CarStatusData' structure, tyreVisua

In [2]:

import pickle 
with open("our_weather.pkl", 'rb') as f:
    L = pickle.load(f)

In [3]:
list_js = []
for i in range(len(L)):
    js = {'timestamp': L[i][0],
          'm_packet_format':L[i][1].header.packetFormat,
          'm_game_major_version':L[i][1].header.gameMajorVersion,
          'm_game_minor_version':L[i][1].header.gameMinorVersion, 
          "m_packet_version":L[i][1].header.packetVersion, 
          'm_packet_id':L[i][1].header.packetId, 
          'm_session_UID':L[i][1].header.sessionUID,
          'm_session_time':L[i][1].header.sessionTime, 
          'm_frame_identifier':L[i][1].header.frameIdentifier, 
          'm_player_car_index':L[i][1].header.playerCarIndex, 
          'm_secondary_player_car_index':L[i][1].header.secondaryPlayerCarIndex, 
          'm_weather':L[i][1].weather, 
          'm_track_temperature':L[i][1].trackTemperature, 
          'm_air_temperature':L[i][1].airTemperature, 
          'm_total_laps':L[i][1].totalLaps, 
          'm_track_length':L[i][1].trackLength, 
          'm_session_type':L[i][1].sessionType, 
          'm_track_id':L[i][1].trackId, 
          'm_formula':L[i][1].formula, 
          'm_session_time_left':L[i][1].sessionTimeLeft, 
          'm_session_duration':L[i][1].sessionDuration, 
          'm_pit_speed_limit':L[i][1].pitSpeedLimit, 
          "m_game_paused":L[i][1].gamePaused, 
          "m_is_spectating":L[i][1].isSpectating, 
          "m_spectator_car_index":L[i][1].spectatorCarIndex, 
          "m_sli_pro_native_support":L[i][1].sliProNativeSupport,
         "m_weather_forecast_samples": []}
    for j in L[i][1].weatherForecastSamples:
        js["m_weather_forecast_samples"].append({"m_weather_forecast_sample_m_session_type":j.sessionType,
                                                "m_time_offset":j.timeOffset,
                                                "m_weather_forecast_sample_m_weather":j.weather,
                                                "m_track_temperature":j.trackTemperature,
                                                 "m_air_temperature":j.airTemperature
                                                })
    list_js.append(js)

In [5]:
import json
json.dump(list_js, open("our_weather.json","w"))